# Multiple Inheritance
In Python it is **not** much more complex than single inheritance.

Remember to use **isinstance() and issubclass()**

In [1]:
help(isinstance)

Help on built-in function isinstance in module builtins:

isinstance(obj, class_or_tuple, /)
    Return whether an object is an instance of a class or of a subclass thereof.
    
    A tuple, as in ``isinstance(x, (A, B, ...))``, may be given as the target to
    check against. This is equivalent to ``isinstance(x, A) or isinstance(x, B)
    or ...`` etc.



In [2]:
#Test it
isinstance(3, int)

True

In [3]:
isinstance("hello", str)

True

In [5]:
isinstance(4.763, bytes)

False

In [8]:
# isinstance accepts tuple of types for the second argument and returns true if any are true
x = []
#x is a list
isinstance(x, (float, dict, list))

True

In [20]:
class SimpleList:
    """
    Sorted List example
    """
    def __init__(self, items):
        self._items = items
        
    def add(self, item):
        self._items.append(item)
    
    def rem(self, items):
        self._items.remove(items)
    
    def __getitem__(self, index):
        return self._items[index]
    
    def sort(self):
        self._items.sort()
        
    def __len__(self):
        return len(self._items)
    
    def __repr__(self):
        return "SimpleList({!r})".format(self._items)
    

In [21]:
class SortedList(SimpleList):
    def __init__(self, items=()):
        super().__init__(items)
        self.sort()
        
    # overwrite add method
    def add(self, item):
        super().add(item)
        self.sort()
    
    def __repr__(self):
        return "SortedList({!r})".format(list(self))
    

In [26]:
#Test it
s1 = SortedList([4,5,2,99,11,23,54,12,19,75])
s1

SortedList([2, 4, 5, 11, 12, 19, 23, 54, 75, 99])

In [27]:
s1.add(-6)
s1

SortedList([-6, 2, 4, 5, 11, 12, 19, 23, 54, 75, 99])

In [44]:
class IntList(SimpleList):
    # call init on every item in the list.
    def __init__(self, items=()):
        for x in items:
            self._validate(x)
        super().__init__(items)
        
    
    @staticmethod
    def _validate(x):
        if not isinstance(x, int):
            raise TypeError("'" + x + "' is not an integer.")
    
    def add(self, item):
        """ Validate the item is an integer before you add it to the list."""
        self._validate(item)
        super().add(item)
        
        
    def __repr__(self):
        return "IntList({!r})".format(list(self))
    

In [45]:
s2 = IntList([1,6,2,9,7,2,8,"hi", 4])

TypeError: 'hi' is not an integer.

In [46]:
s2 = IntList([1,6,2,9,7,2,8,4])
s2

IntList([1, 6, 2, 9, 7, 2, 8, 4])

In [47]:
s2.add(5)
s2

IntList([1, 6, 2, 9, 7, 2, 8, 4, 5])

In [48]:
s2.add("John")

TypeError: 'John' is not an integer.

## issubclass()
> help(issublass)

In [49]:
issubclass(IntList, SimpleList)

True

In [50]:
issubclass(SortedList, IntList)

False

## Multiple Inheritance
Define a class with more than one base class. <br>
This is not a universal functionality of Object Oriented Langauges. Java doesn't, C++ does.<br>

Be careful when using multiple inheritance.
Note: It could lead to some complicated situations.
 - What if more than one base class defines a particular method
 
Python has a relatively simple and understandable system for **multiple inheritance**

Syntax: **class SubClass(Base1, Base2, ...)**
 - Subclasses inherit methods from ALL bases
 - without conflict, names resolve in the obvious way.
 - **Method Resolution Order (MRO)** determines name lookup in all cases.
 

In [51]:
class SortedIntList(IntList, SortedList):
    def __repr__(self):
        return "SortedIntList({!r})".format(list(self))

In [53]:
# test it
sil = SortedIntList([42, 2, 17, 9])
sil

SortedIntList([2, 9, 17, 42])

In [54]:
# test it
sil2 = SortedIntList([42, 2, "no", 9])
sil2

TypeError: 'no' is not an integer.

In [55]:
# Add method maintains both, toe sorting and type constraints
sil.add(-12)
sil

SortedIntList([-12, 2, 9, 17, 42])

In [56]:
sil.add("john")

TypeError: 'john' is not an integer.

** How does Python know which add() to call?**<br>
** How does Python maintain both constraints?**<br>

The answer to both questions is the **Resolution Method Order**.<br>
**MRO and super()**

If a class:
 - has **multiple** base classes
 - defines **no initializer** then **only** the initializer of the **first** base class is automatically called.

In [58]:
class Base1:
    def __init__(self):
        print("Base1.__init__")
        
class Base2:
    def __init__(self):
        print("Base2.__init__")
    
class sub(Base1, Base2):
    pass


In [59]:
s = sub()

Base1.__init__


Through the use of **super()** we could design these class such that both Base1 and Base2 dunder-inits are called automatically

**\_\_bases\_\_**: A tuple of bases classes


In [60]:
SortedIntList.__bases__

(__main__.IntList, __main__.SortedList)

In [62]:
IntList.__bases__

(__main__.SimpleList,)

## Method Resolution Order(MRO)
Ordering that determines method name lookup

 - Methods may be defined in multiple places
 - MRO is an ordering of the inheritance graph
 - It is quite simple!
 
Let's look at an example, but let's look at where our MRO is stored first.<br>
**\_\_mro\_\_** returns a list and **mro()** returns a tuple<br>
page 68 in our book shows a graph to better illustrate the MRO

In [68]:
SortedIntList.__mro__

(__main__.SortedIntList,
 __main__.IntList,
 __main__.SortedList,
 __main__.SimpleList,
 object)

In [69]:
SortedIntList.mro()

[__main__.SortedIntList,
 __main__.IntList,
 __main__.SortedList,
 __main__.SimpleList,
 object]

## How is MRO used?
**obj.method()**<br>
class SomeClass
1. Instance of:
    - Base1!
    - Base2!
    - Base3! yes. A hit here.
    - Base4!
2. MRO
    - match!
    - Base3.method(obj)
3. resolve to
    - ex. 19
    

In [75]:
class A:
    def func(self):
        return "A.func"

class B(A):
    def func(self):
        return "B.func"

class C(A):
    def func(self):
        return "C.func"
class D(C,B):
    pass

In [76]:
D.mro()

[__main__.D, __main__.C, __main__.B, __main__.A, object]

In [78]:
#What should you expect when typing d.func()? I'd assume it will run C.func.
d = D()
d.func()

'C.func'

In [79]:
# Change the order of base classes
class D(B,C):
    pass

In [80]:
d = D()
d.func()

'B.func'

In [83]:
D.__mro__

(__main__.D, __main__.B, __main__.C, __main__.A, object)

In [85]:
SortedIntList.mro()

[__main__.SortedIntList,
 __main__.IntList,
 __main__.SortedList,
 __main__.SimpleList,
 object]

This is properly maintaining both the sorting constraint and then the type constraint of both SortedList and IntList.<br>
**how is intList.add() deferring to SortedList.add()?** The answer is how super() actually works.


### How does Python calculate the MRO?

C3: Algorithm for calculating MRO in Python
 - Subclasses come **before** base classes.
 - Base class order from class definition is **preserved**.
 -First two wuqlities are preserved **no matter** where you start in the inheritance graph.
 
Note: Not all inheritance declarations are allowed.


In [88]:
class A:
    def func(self):
        return "A.func"

class B(A):
    def func(self):
        return "B.func"

class C(A):
    def func(self):
        return "C.func"
class D(B, A, C): #order matters alot. how does it get the mro of C when A has a lower priority mro
    pass

TypeError: Cannot create a consistent method resolution
order (MRO) for bases A, C

Note: Since both B and C inherit from A, they **must** come before A.
    
C3 cannot put A both before and after C.

## The built-in super() function

Todo: (come back to it the last day of class if time)